In [ ]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [ ]:
df = pd.read_csv('merged_scraped_cleaned.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#add new columns for Transit Screen API info

In [ ]:
df['ts_response'] = np.nan
df['mobility_score'] = np.nan
df['carshare'] = np.nan
df['bikeshare'] = np.nan
df['ridehailing'] = np.nan
df['masstransit'] = np.nan

In [ ]:
base_url = 'http://mobilityscore-dev.transitscreen.io/api/v1/locations.json?&debug=true'
key = keys.ts

for i, row in df[(df['ts_response'].isnull()==True)
                 #& (df['id']=='0061a00000GcQdL') #for test
                ].iterrows():
    #print(row.id, row.name, row.lat, row.lng)
    try:
        url = base_url +'&key='+key+'&geojson=true&coordinates='+str(row.lat)+','+str(row.lng)
        #(print(url))
        response = requests.get(url) #wait up to 10 seconds
        data = json.loads(response.text)
        full_response = data['data']
        mobility_score = full_response['mobilityScore']['score']
        #print (mobility_score)
        raw_scores = full_response['mobilityScore']['rawScoreBreakdown']
        #print (raw_scores)
        carshare = raw_scores['carshare']
        bikeshare = raw_scores['bikeshare']
        ridehailing = raw_scores['ridehailing']
        masstransit = raw_scores['masstransit']
        #print(carshare, bikeshare, ridehailing, masstransit)
        df.loc[df['id']==row.id, 'ts_response'] = data
        df.loc[df['id']==row.id, 'mobility_score'] = mobility_score
        df.loc[df['id']==row.id, 'carshare'] = carshare
        df.loc[df['id']==row.id, 'bikeshare'] = bikeshare
        df.loc[df['id']==row.id, 'ridehailing'] = ridehailing
        df.loc[df['id']==row.id, 'masstransit'] = masstransit
    except:
        print('error on', i, raw_scores)
    if i%100 == 0:
        df.to_csv('model_data.csv', index=False)
        print (i, raw_scores)


In [ ]:
df[df.id == '0061a00000GcQdL']

In [ ]:
df.to_csv('model_data.csv', index=False)

In [ ]:
df.to_csv('model_data_transit_scores_only.csv', index=False)

In [ ]:
df = pd.read_csv('model_data.csv')

In [ ]:
df.head(100)